# 0. Preliminaries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import platform
import re

In [3]:
'''
if platform.system() == 'Darwin':
    user = 'joewandy'
elif platform.system() == 'Windows':
    user = 'joewa'
    
user_vimms = '/Users/%s/Work/git/vimms' % user
'''
#user_vimms = os.path.join("C:\\", "Users", "mcbrider5002", "Desktop", "Workspace", "phd", "peak_picking", "vimms")
user_vimms = "vimms"
sys.path.append(user_vimms)

In [4]:
from vimms.Common import (
    POSITIVE, ROI_TYPE_SMART, ROI_EXCLUSION_WEIGHTED_DEW,
    set_log_level_warning,
)

from vimms.Roi import RoiBuilderParams, SmartRoiParams
from vimms.Experiment import ExperimentCase, Experiment
from vimms.Controller import TopNController

 C:\Users\mcbrider5002\anaconda3\lib\site-packages\psims\mzmlb\writer.py:15: UserWarning:hdf5plugin is missing! Only the slower GZIP compression scheme will be available! Please install hdf5plugin to be able to use Blosc.


In [5]:
ionisation_mode = POSITIVE
pbar = False
set_log_level_warning()

1

### Get seed data

In [6]:
def match_files(data_dir, regex):
    pattern = re.compile(regex)
    files = (
        (int(pattern.match(fname).group(1)), fname)
        for fname in os.listdir(data_dir)
        if not pattern.match(fname) is None
    )

    return [
        os.path.join(data_dir, fname) 
        for _, fname in sorted(files, key=lambda p: p[0])
    ]

In [7]:
#multi_dir = "/Users/%s/University of Glasgow/Vinny Davies - CLDS Metabolomics Project/Experimental_Results/20220706_DDAvsDIA" % user
multi_dir = "20220719_multi_samples_main"
multibeer_dir = os.path.join(multi_dir, "results_4")

In [8]:
multibeer_fullscans = match_files(multibeer_dir, r"fullscan_beer([0-9]+)_0.mzML")

### Specify parameters

In [9]:
num_workers = 8 #can't always use number of physical cores because of memory constraints

In [10]:
#coverage params
'''
experiment_params = {
    "topN_params": {
        "ionisation_mode" : ionisation_mode,
        "N" : 10,
        "isolation_width" : 1,
        "min_ms1_intensity" : 5000,
        "mz_tol" : 10,
        "rt_tol" : 60
    },
    
    "roi_params" : {
        "min_roi_length_for_fragmentation" : 0,
        "roi_params" : RoiBuilderParams(
                            min_roi_intensity=0,
                            min_roi_length=3,
                       )
    },
    
    "non_overlap_params": {
    },
    
    "smartroi_params": {
        "rt_tol" : 15,
        "smartroi_params" : SmartRoiParams(
                                reset_length_seconds=1E6,
                                intensity_increase_factor=3,
                                drop_perc=0.001
                            )  
    },
    
    "weighteddew_params": {
        "rt_tol": 120,
        "exclusion_method": ROI_EXCLUSION_WEIGHTED_DEW,
        "exclusion_t_0": 60
    }    
}
'''

#intensity params
experiment_params = {
    "topN_params": {
        "ionisation_mode" : ionisation_mode,
        "N" : 20,
        "isolation_width" : 1,
        "min_ms1_intensity" : 5000,
        "mz_tol" : 10,
        "rt_tol" : 60
    },
    
    "roi_params" : {
        "min_roi_length_for_fragmentation" : 0,
        "roi_params" : RoiBuilderParams(
                            min_roi_intensity=0,
                            min_roi_length=3,
                       )
    },
    
    "non_overlap_params": {
    },
    
    "smartroi_params": {
        "rt_tol" : 15,
        "smartroi_params" : SmartRoiParams(
                                reset_length_seconds=1E6,
                                intensity_increase_factor=3,
                                drop_perc=0.001
                            )  
    },
    
    "weighteddew_params": {
        "rt_tol": 60,
        "exclusion_method": ROI_EXCLUSION_WEIGHTED_DEW,
        "exclusion_t_0": 1
    }    
}

min_rt = 0
max_rt = 1440
scan_duration_dict = { #CHANGED FROM QCB
    1: 0.59,
    2: 0.19
}
point_noise_threshold = 0 #CHANGED FROM QCB

### Specify controllers to run

In [11]:
topN_params = experiment_params["topN_params"]
roi_params = {**topN_params, **experiment_params["roi_params"]}
non_overlap_params = {**roi_params, **experiment_params["non_overlap_params"]}

cases = [
    ("topN", topN_params),
    ("topN_RoI", roi_params),
    ("topN_exclusion", topN_params),
    ("topNEx", non_overlap_params),
    ("hard_roi_exclusion", non_overlap_params),
    ("intensity_roi_exclusion", non_overlap_params),
    ("non_overlap", non_overlap_params),
    ("intensity_non_overlap", non_overlap_params)
]

no_smartroi = ["topN", "topN_RoI", "topN_exclusion"]
#run_smartroi = []
run_smartroi = [name for name, _ in cases if not name in no_smartroi]
#run_weighteddew = []
run_weighteddew = [name for name, _ in cases if not name in no_smartroi]

In [12]:
new_cases = []
for controller_type, params in cases:
    new_cases.append(
        (controller_type, controller_type, params)
    )
    
    if(controller_type in run_smartroi):
        new_name = controller_type + "_smartroi"
        new_params = {**params, **experiment_params["smartroi_params"]}
        new_cases.append(
            (controller_type, new_name, new_params)
        )
        
    if(controller_type in run_weighteddew):
        new_name = controller_type + "_weighteddew"
        new_params = {**params, **experiment_params["weighteddew_params"]}
        new_cases.append(
            (controller_type, new_name, new_params)
        )
        
cases = new_cases

In [13]:
for controller_type, name, params in cases:
    print(f"NAME: {name}")
    print(f"CONTROLLER TYPE: {controller_type}")
    print(f"PARAMS: {params}")
    print()

NAME: topN
CONTROLLER TYPE: topN
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60}

NAME: topN_RoI
CONTROLLER TYPE: topN_RoI
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60, 'min_roi_length_for_fragmentation': 0, 'roi_params': {'mz_tol': 10, 'min_roi_length': 3, 'min_roi_intensity': 0, 'at_least_one_point_above': 0, 'start_rt': 0, 'stop_rt': 100000.0, 'max_gaps_allowed': 0}}

NAME: topN_exclusion
CONTROLLER TYPE: topN_exclusion
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60}

NAME: topNEx
CONTROLLER TYPE: topNEx
PARAMS: {'ionisation_mode': 'Positive', 'N': 20, 'isolation_width': 1, 'min_ms1_intensity': 5000, 'mz_tol': 10, 'rt_tol': 60, 'min_roi_length_for_fragmentation': 0, 'roi_params': {'mz_tol': 10, 'min_roi_length': 3, 'min_roi_intensity': 0, 'at_least_on

# 1. Same Beer Repeated Multiple Times

In [14]:
repeat = 20
out_dir = "new_same_beer"

same_beer_exp = Experiment()
same_beer_exp.add_cases(
    ExperimentCase(controller_type, multibeer_fullscans[:1] * repeat, params, name=name, pickle_env=True)
    for controller_type, name, params in cases
)
same_beer_exp.run_experiment(
    out_dir,
    min_rt=min_rt,
    max_rt=max_rt,
    ionisation_mode=ionisation_mode,
    scan_duration_dict=scan_duration_dict,
    overwrite_keyfile=False,
    point_noise_threshold=point_noise_threshold,
    chem_noise_threshold=experiment_params["topN_params"]["min_ms1_intensity"] * 0.5, #filter low intensity signal for memory
    num_workers=num_workers
)

Creating Chemicals...

Running Experiment of 18 cases...


# 2. Multiple Different Beers

In [15]:
repeat = 6
out_dir = "new_different_beer"

diff_beer_exp = Experiment()
diff_beer_exp.add_cases(
    ExperimentCase(controller_type, multibeer_fullscans[:repeat], params, name=name, pickle_env=False)
    for controller_type, name, params in cases
)
diff_beer_exp.run_experiment(
    out_dir,
    min_rt=min_rt,
    max_rt=max_rt,
    ionisation_mode=ionisation_mode,
    scan_duration_dict=scan_duration_dict,
    overwrite_keyfile=False,
    point_noise_threshold=point_noise_threshold,
    chem_noise_threshold=experiment_params["topN_params"]["min_ms1_intensity"] * 0.5, #filter low intensity signal for memory
    num_workers=num_workers
)

Creating Chemicals...

Running Experiment of 18 cases...


# 3. Repeating Different Beers

In [16]:
bio_repeat = 6
tech_repeat = 4
out_dir = "new_repeated_different_beer"

rep_diff_beer_exp = Experiment()
rep_diff_beer_exp.add_cases(
    ExperimentCase(controller_type, multibeer_fullscans[:bio_repeat] * tech_repeat, params, name=name, pickle_env=True)
    for controller_type, name, params in cases
)
rep_diff_beer_exp.run_experiment(
    out_dir,
    min_rt=min_rt,
    max_rt=max_rt,
    ionisation_mode=ionisation_mode,
    scan_duration_dict=scan_duration_dict,
    overwrite_keyfile=True,
    point_noise_threshold=point_noise_threshold,
    chem_noise_threshold=experiment_params["topN_params"]["min_ms1_intensity"] * 0.5, #filter low intensity signal for memory
    num_workers=num_workers
)

Creating Chemicals...

Running Experiment of 18 cases...
